<a href="https://colab.research.google.com/github/JuanLeonelSantamariaMena/Sim_PA_FEMFG/blob/main/FEMFG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
#import pandas as pd
import time
import plotly.graph_objects as go
import plotly.express as px

class WaveSimulation_PA:
    def __init__(self,
                 E0 = 10e-3,
                 Cp = 4148e3,
                 taup = 10e-9,
                 b = 6.9e-5,
                 c = 1440,
                 d0 = 50e-6,
                 size_source_pa = 1e-3,
                 n_source_pa = 30,
                 size_array_sensor = 4e-2,
                 n_array_sensor = 256,
                 origin_source = [0,5e-3,5e-3],
                 origin_array_sensor = [0,0,0],
                 time_start = 0e-6,
                 time_end = 2.56e-5,
                 delta_time = 1e-7,
                 delay = 0,
                 absorption=0.566,
                 model='d0'):
        """
        E0:                     Energia relacionada al láser ()
        Cp:                     Capacida calorifica (kJ/(kg °C))
        b:                      Coeficiente de dilatación
        taup:                   Ancho de pulso
        c:                      Velocidad del medio (m/s)
        d0:                     **Distancia entre la fuente puntual y el sensor (m)
        size_source:            Tamaño de la fuente puntual (m)
        n_source:               Número de fuentes puntuales
        size_array_sensor:      Tamaño del arreglo de sensores (m)
        n_array_sensor:         Número de sensores
        origin_source:          Origen de la fuente puntual: x,y,z (m)
        origin_array_sensor:    Origen del arreglo de sensores: x,y,z (m)
        model:                  modelo presion tau ó d0
        """
        self.E0 = E0
        self.Cp = Cp
        self.b = b
        self.c = c
        self.taup = taup
        #self.d0 = size_source_pa*(n_source_pa*0.0013)*2
        self.d0 = (size_source_pa/n_source_pa)*(n_source_pa*0.113)
        if model=='d0':
            self.size_source_pa = (size_source_pa*(1.215))/2 #1.215
        elif model=='tau':
            self.size_source_pa = size_source_pa/2
        self.time = np.arange(time_start,time_end,delta_time)
        self.n_source_pa = n_source_pa
        self.size_array_sensor = size_array_sensor/2
        self.n_array_sensor = n_array_sensor//2
        self.origin_source = origin_source
        self.origin_array_sensor = origin_array_sensor
        self.delay=delay
        self.absorption=absorption
        self.model = model
    def P_d0(self, R, t):
        """
          Esta funcion calcula la presión de la onda dentro de un espacio de tiempo t y
        una distancia total R, a partir del modelo d_0.
        """
        p0 = (4 * self.E0 * self.b * self.c**2) / ((np.pi**(3/2)) * self.Cp * self.d0)
        P = p0 * ((self.c * t - R) / R) * np.exp(-((self.c * t - R) / (self.d0 / 2))**2)
        if self.delay !=0:
            t1=(t[-1]-R/self.c)//self.delay
            for i in range(int(t1)+1):
                amplitud=1.7273*np.exp(-self.absorption*i)
                P1 = p0 * ( (self.c*(t-self.delay*i) - R) /R ) * np.exp( -( (self.c*(t-self.delay*i) - R) / (self.d0/2) )**2 )*amplitud
                P+=P1
        return P
    def P_tau(self,R,t):
        p0 = (8 * self.E0 * self.b * self.c)/( (np.pi**(3/2))*self.taup*self.Cp*R)
        P = p0 * ( (self.c*t - R) /(self.c*(self.taup**2)) ) * np.exp( -( (self.c*t - R) / (self.c*self.taup) )**2 )
        if self.delay != 0:
          P1 = p0 * ( (self.c*(t-self.delay) - R) /(self.c*(self.taup**2)) ) * np.exp( -( (self.c*(t-self.delay) - R) / (self.c*self.taup) )**2 )
          return P+P1
        else :
          return P
    def simul_sphera(self):
        """
          Se simula un arreglo de cordenadas de fuentes puntuales con una forma esferica
        de forma base se genera una serie de cordenas que posterior se podra re-escalar
        """
        n=self.n_source_pa/2
        x,y,z = np.arange(-n,n+1),np.arange(-n,n+1),np.arange(-n,n+1)
        mx,my,mz = np.meshgrid(x,y,z)
        m = np.array([mx,my,mz])
        d = np.sqrt(mx**2+my**2+mz**2)
        a = np.argwhere(d<=n).T
        m_e = m.T[a[0],a[1],a[2]]
        m_e = m_e.T/(n-1)
        self.source_pa = np.array([m_e[0],m_e[1],m_e[2]]).T*self.size_source_pa
        self.source_pa = self.origin_source + self.source_pa
        return self.source_pa
    def simul_sphera_special(self):
        n=self.n_source_pa/2
        x,y,z = np.arange(-n,n+1),np.arange(-n,n+1),np.arange(-n,n+1)
        mx,my,mz = np.meshgrid(x,y,z)
        m = np.array([mx,my,mz])
        d = np.sqrt(mx**2+my**2+mz**2)
        a = np.argwhere((d<=n) & (d>(n-(n*0.06)))).T
        b = np.argwhere(d<=(n-(n*0.12))).T
        m_a = m.T[a[0],a[1],a[2]]
        m_b = m.T[b[0],b[1],b[2]]
        m_a = m_a.T/(n-1)
        m_b = m_b.T/(n-1)
        self.source_pa = np.array([m_a[0],m_a[1],m_a[2]]).T*self.size_source_pa
        self.source_pa = self.origin_source + self.source_pa
        self.source_pa_b = np.array([m_b[0],m_b[1],m_b[2]]).T*self.size_source_pa
        self.source_pa_b = self.origin_source + self.source_pa_b
        self.source_pa = self.source_pa.T
        self.source_pa_b = self.source_pa_b.T
        return self.source_pa, self.source_pa_b
    def simul_arraylinearsensor(self):
        """
        Se simula un arreglo de coordenadas de fuentes puntuales con una forma esférica
        de forma base se genera una serie de coordenadas que posteriormente se podrá
        reescalar.
        """
        n=self.n_array_sensor
        x_s,y_s,z_s = np.linspace(-n,n+1,int(n*2)),np.zeros(int(n*2)),np.zeros(int(n*2))
        self.array_sensor =  (np.array([x_s,y_s,z_s])/(n-1)).T*self.size_array_sensor + self.origin_array_sensor
        return self.array_sensor
    def simul_arraycircularsensor(self):
        """
        Se simula un arreglo de coordenadas de fuentes puntuales con una forma esférica
        de forma base se genera una serie de coordenadas que posteriormente se podrá
        reescalar.
        """
        n=self.n_array_sensor/2
        x_s, y_s, z_s = np.arange(-n,n+1), np.arange(-n,n+1), np.zeros(int(n*2))
        mx, my, z_s = np.meshgrid(x_s,y_s,z_s)
        mx, my ,z_s = mx.reshape(-1), my.reshape(-1), z_s.reshape(-1)
        d=np.sqrt(mx**2+my**2)
        a=np.argwhere(d<n).T

        self.array_sensor = (np.array([mx[a],my[a],z_s[a]]).T/(n-1) * self.size_array_sensor).squeeze(axis = 1) + self.origin_array_sensor
        return self.array_sensor
    def simulacion2imag(self):
        """
        Se simula un arreglo de coordenadas de fuentes puntuales con una forma esférica
        de forma base se genera una serie de coordenadas que posteriormente se podrá
        reescalar.
        """
        self.source_pa = self.simul_sphera().T
        self.array_sensor = self.simul_arraylinearsensor()
        image=[]
        for i in range(len(self.array_sensor)):
          distance = ((self.source_pa[0]-self.array_sensor[i][0])**2+(self.source_pa[1]-self.array_sensor[i][1])**2+(self.source_pa[2]-self.array_sensor[i][2])**2)**(1/2)
          presion = 0
          for j in range(len(distance)):
              if self.model == 'd0':
                  presion += self.P_d0(distance[j],self.time)
              elif self.model == 'tau':
                  presion += self.P_tau(distance[j], self.time)
          #presion = presion / np.max(np.abs(presion))
          image.append(presion)
        image = np.array(image)
        return image
    def simula2senal(self):
        """
          Se simula un arreglo de coordenadas de fuentes puntuales con una forma esférica
          y el sensor es un arreglo de sensores puntuales.
        """
        if self.model =='d0':
            self.source_pa,self.source_pa_b=self.simul_sphera_special()
        elif self.model =='tau':
            self.source_pa = self.simul_sphera().T
        if self.n_array_sensor == 1:
            presion = 0
            distance = ((self.source_pa[0]-0)**2+(self.source_pa[1]-0)**2+(self.source_pa[2]-0)**2)**(1/2)
            if self.model =='d0':
                distance_b = ((self.source_pa_b[0]-0)**2+(self.source_pa_b[1]-0)**2+(self.source_pa_b[2]-0)**2)**(1/2)
                for j in range(len(distance_b)):
                        presion += self.P_d0(distance_b[j],self.time)
            for j in range(len(distance)):
                if self.model == 'd0':
                    self.d0 = self.size_source_pa/1.2
                    #presion += self.P_d0(distance[j],self.time)*0
                    self.d0 = self.size_source_pa*1.2
                elif self.model == 'tau':
                    presion += self.P_tau(distance[j], self.time)
            presion = presion / np.max(np.abs(presion))
            presion += presion
            return presion
        else :
            self.array_sensor = self.simul_arraycircularsensor().T
            presion = np.zeros(len(self.time))
            for i in range(len(self.array_sensor)):
              distance = ((self.source_pa[0]-self.array_sensor[i][0])**2+(self.source_pa[1]-self.array_sensor[i][1])**2+(self.source_pa[2]-self.array_sensor[i][2])**2)**(1/2)
              presion = 0
              for j in range(len(distance)):
                presion += self.P_d0(distance[j],self.time)
              # presion = presion / np.max(np.abs(presion))
              presion += presion
            return presion